In [2]:
!pip install gplearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import gplearn as gpl 
from gplearn.genetic import SymbolicRegressor   
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/regression/nasa_turbofan/fd001/train_FD001.csv')
X_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/regression/nasa_turbofan/fd001/test_FD001.csv')
Y_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/regression/nasa_turbofan/fd001/test_rul_FD001.csv')
train

,unit_nr,time_cycles,setting_1,setting_2,s_2,s_3,s_4,s_7,s_8,s_9,s_11,s_12,s_13,s_15,s_17,s_20,s_21,RUL,breakdown
0,1,1,-0.0007,-0.0004,641.82,1589.70,1400.60,554.36,2388.06,9046.19,47.47,521.66,2388.02,8.4195,392,39.06,23.4190,191,0
1,1,2,0.0019,-0.0003,642.15,1591.82,1403.14,553.75,2388.04,9044.07,47.49,522.28,2388.07,8.4318,392,39.00,23.4236,190,0
2,1,3,-0.0043,0.0003,642.35,1587.99,1404.20,554.26,2388.08,9052.94,47.27,522.42,2388.03,8.4178,390,38.95,23.3442,189,0
3,1,4,0.0007,0.0000,642.35,1582.79,1401.87,554.45,2388.11,9049.48,47.13,522.86,2388.08,8.3682,392,38.88,23.3739,188,0
4,1,5,-0.0019,-0.0002,642.37,1582.85,1406.22,554.00,2388.06,9055.15,47.28,522.19,2388.04,8.4294,393,38.90,23.4044,187,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,643.49,1597.98,1428.63,551.43,2388.19,9065.52,48.07,519.49,2388.26,8.4956,397,38.49,22.9735,4,0
20627,100,197,-0.0016,-0.0005,643.54,1604.50,1433.58,550.86,2388.23,9065.11,48.04,519.68,2388.22,8.5139,395,38.30,23.1594,3,0
20628,100,198,0.0004,0.0000,643.42,1602.46,1428.18,550.94,2388.24,9065.90,48.09,520.01,2388.24,8.5646,398,38.44,22.9333,2,0
20629,100,199,-0.0011,0.0003,643.23,1605.26,1426.53,550.68,2388.25,9073.72,48.39,519.67,2388.23,8.5389,395,38.29,23.0640,1,0


In [5]:
to_remove = ['unit_nr', 'breakdown', 'setting_1', 'setting_2']
output_column = ['RUL']
categorical = []
numerical = [col for col in train.columns if col not in categorical + output_column + to_remove]

In [6]:
features = categorical + numerical
 
# Creating the X, T and E inputs
X_train, Y_train = train[features], train[output_column].values
#X_test = X_test[features]

cat_enc_d = {}
for cat in categorical:
    OE = OneHotEncoder(sparse=False, drop='if_binary')
    train_ohe = OE.fit_transform(X_train[[cat]])
    test_ohe = OE.transform(X_test[[cat]])
    for i in range(train_ohe.shape[1]):
        c = OE.categories_[0][i]
        X_train[cat+'_'+str(c)] = train_ohe[:, i]
        X_test[cat+'_'+str(c)] = test_ohe[:, i]
    cat_enc_d[cat] = OE

X_train.drop(columns=categorical, inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(X_train[numerical])
scaled_test = scaler.transform(X_test[numerical])
X_train[numerical], X_test[numerical] = scaled_train, scaled_test

y_scaler = MinMaxScaler(feature_range=(0, 1))
Y_train = y_scaler.fit_transform(Y_train) + 1e-10

X_train.shape, X_test.shape

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



((20631, 14), (13096, 17))

In [8]:
def power_f(x, d):
    d = np.round(np.clip(d, 1, 5)).astype(int)
    return np.power(x, d)

def tanh_f(x, a, b):
    return np.tanh(a*x+b)


def _r2score(y, y_pred,w=None):
    return r2_score(y, y_pred)

power = gpl.functions.make_function(function=power_f, name='pow', arity=2, wrap=True)
tanh = gpl.functions.make_function(function=tanh_f, name='tanh', arity=3, wrap=True)

r2score = gpl.fitness.make_fitness(function=_r2score, greater_is_better=True)

function_set = [power, tanh, 'add', 'sub', 'mul', 'div', 'sqrt', 'log', 'min', 'max', 'sin', 'abs', 'neg', 'cos']

model = SymbolicRegressor(population_size = 2000, tournament_size=100,const_range=(-10,10),
                          generations = 1000, stopping_criteria=0.,
                          function_set = function_set, metric='mean absolute error',
                          p_crossover=0.6, p_subtree_mutation=0.25,
                          p_hoist_mutation=0.05, p_point_mutation=0.1, low_memory=True,
                          verbose = 1, random_state = None, n_jobs = -1)

model.fit(X_train.values, Y_train.ravel())
print(model._program)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    14.00      4.08147e+06       18         0.106673              N/A     79.07m
   1     7.44         0.464968        5         0.100778              N/A     47.57m
   2     4.06         0.403792       10        0.0999122              N/A     56.42m
   3     5.19          1.32374       11        0.0930964              N/A     64.38m
   4     5.58         0.880841       11        0.0930964              N/A     47.42m
   5    10.20         0.429363       13        0.0915647              N/A     92.72m
   6    10.84         0.400053       13        0.0915647              N/A    161.61m
   7    10.61         0.426618       13        0.0920366              N/A     68.16m
   8    10.82         0.378652       13        0.0915647              N/A  

In [9]:
pred = y_scaler.inverse_transform(model.predict(X_train)[:,np.newaxis]).ravel()
Y_train = y_scaler.inverse_transform(Y_train).ravel()
r2_score(Y_train, pred), mean_squared_error(Y_train, pred, squared=False),  mean_absolute_error(Y_train, pred)

(0.6538783323120166, 40.52315491115029, 28.839501565995917)

In [10]:
last_cycle = X_test.groupby('unit_nr').last()[features]
pred = y_scaler.inverse_transform(model.predict(last_cycle)[:,np.newaxis]).ravel()
r2_score(Y_test, pred), mean_squared_error(Y_test, pred, squared=False),  mean_absolute_error(Y_test, pred)

(0.5898615041907311, 26.61307385858976, 20.692982550821785)

In [16]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=last_cycle.index, y=Y_test.values.ravel(), mode='lines', name='Y_test'))
fig.add_trace(go.Scatter(x=last_cycle.index, y=pred, mode='lines', name='PRED'))
fig.add_trace(go.Scatter(x=last_cycle.index, y=np.abs(Y_test.values.ravel()-pred), mode='lines', name='DIFF'))

fig.show()

In [15]:
Y_test.values.ravel()

array([112,  98,  69,  82,  91,  93,  91,  95, 111,  96,  97, 124,  95,
       107,  83,  84,  50,  28,  87,  16,  57, 111, 113,  20, 145, 119,
        66,  97,  90, 115,   8,  48, 106,   7,  11,  19,  21,  50, 142,
        28,  18,  10,  59, 109, 114,  47, 135,  92,  21,  79, 114,  29,
        26,  97, 137,  15, 103,  37, 114, 100,  21,  54,  72,  28, 128,
        14,  77,   8, 121,  94, 118,  50, 131, 126, 113,  10,  34, 107,
        63,  90,   8,   9, 137,  58, 118,  89, 116, 115, 136,  28,  38,
        20,  85,  55, 128, 137,  82,  59, 117,  20])